In [58]:
import psycopg2
import geopandas as gpd
import pandas as pd
from jinja2 import Environment, FileSystemLoader
from config import config

# CONEXIÓN a la BASE DE DATOS

params = config()
conn = psycopg2.connect(**params)

zona = 'losar'

# CONDICIONES SINÓPTICAS

sql = '''SELECT * 
        FROM incendios_proximos_ze 
        ORDER by tot desc
        '''

incendios_cs = gpd.read_postgis(sql, conn)[:20]

tabla = incendios_cs[['idpif', 
                      'deteccion', 
                      'municipio', 
                      'paraje', 
                      'superficiearboladatotal', 
                      'superficinoarboladatotal', 
                      'tot']]

tabla.columns = ['Id PIF', 'fecha', 'municipio', 'paraje', 'sup arbolada', 'sup no arbolada', 'sup total']

condiciones_sinopticas = []

for key, incendio in incendios_cs.iterrows():
    condiciones_sinopticas.append({
        'fecha': incendio['deteccion'],
        'municipio': incendio['municipio'],
        'paraje': incendio['paraje'],
        'idcausa': incendio['idcausa'],
        'idpif': incendio['idpif'],
        'sup_arbolada': incendio['superficiearboladatotal'],
        'sup_no_arbolada': incendio['superficinoarboladatotal'],
        'sup_total': incendio['tot'],
    }
    )
    
# SEVERIDAD 

incendios = [{'zona': 'Jerte-Tornavacas', 'idpif': ['1133522', '1143015', '1143273', '574176', '1164099']}, 
             {'zona': 'Villanueva-Madrigal', 'idpif': ['1122656', '1132420', '1133464', '1163320', '1163391']}, 
             {'zona': 'Losar', 'idpif': ['1163407', '569396', '1169767', '574250', '1122672']}]

severidad = []

zonas_dict = {'jerte': 0, 'tornavacas': 0, 'madrigal': 1, 'losar': 2}

severidad = incendios[zonas_dict[zona]]
severidad['incendios'] = []

for n in severidad['idpif']:
    perimetro = gpd.read_postgis("select * from perimetros_utm where idpif = {}".format(n), conn)
    severidad['incendios'].append({'fecha': perimetro.iloc[0]['fecha'].strftime('%d/%m/%Y'), 'idpif': n})
        
# CATASTRO 

catastro = []
sql = '''
    SELECT a.*
    FROM catastro_{} a, zona_estudio_{} b
    WHERE st_intersects(a.geom, b.geom) 
    AND a.tipo = 'R'
    AND st_area(st_intersection(a.geom, b.geom)) > 50000
'''.format(zona, zona)

parcelas = gpd.read_postgis(sql, conn)
for key, parcela in parcelas.iterrows():
    catastro.append({'key': key,
                     'refcat': parcela['refcat'], 
                     'fecha': parcela['fechaalta'], 
#                      'propietario1': parcela['propietario1'], 
#                      'propietario2': parcela['propietario2'], 
#                      'contacto1': parcela['contacto1'], 
#                      'contacto2': parcela['contacto2'], 
#                      'nombre': parcela['nombre'], 
                     'sup': "{:.2f}".format(parcela['area'] / 10000.0)})

# FORESTALES
    
forestales = pd.read_sql("select forestal, contacto from forestales where zona = '{}'".format(zona), conn)
display(forestales)

# HIDROGRAFÍA

rios = gpd.read_postgis('''select 
                row_number() over (order by a.long_km desc) as id_mapa, a.* 
                from rios a, zona_estudio_{} b 
                where st_intersects(a.geom, b.geom)
                and a.long_km < 50'''.format(zona), conn)

# METEO (TODO crear tabla en base de datos para no tener tanto código aquí)
datos_meteo = []
for estacion in ['madrigal', 'piornal']:
    dias_lluvia = pd.read_sql('''
                            select estacion, count(*) as dias_lluvia
                            from meteo_{} 
                            where prectotal > 0 and estacion is not null
                            and estacion != 'invierno_2020_2021'
                            group by estacion 
                            order by estacion'''.format(estacion), conn)
    dias_sin_lluvia_5 = pd.read_sql('''
                            select estacion, count(*) as dsl5
                            from meteo_{} 
                            where dias_sin_lluvia_5 is true and estacion is not null
                            group by estacion 
                            order by estacion'''.format(estacion), conn)
    dias_sin_lluvia_8 = pd.read_sql('''
                            select estacion, count(*) as dsl8
                            from meteo_{} 
                            where dias_sin_lluvia_8 is true and estacion is not null
                            group by estacion 
                            order by estacion'''.format(estacion), conn)
    nieve = pd.read_sql('''
                            select estacion, count(*) as nieve
                            from meteo_{}
                            where prectotal > 0 and tmin < 0 and estacion is not null
                            group by estacion 
                            order by estacion'''.format(estacion), conn)
    heladas = pd.read_sql('''
                            select estacion, count(*) as heladas
                            from meteo_{}
                            where tmin < 0 and estacion is not null
                            group by estacion 
                            order by estacion'''.format(estacion), conn)
    ventana_solana = pd.read_sql('''
                            select estacion, count(*) as vs
                            from meteo_{} 
                            where tmax < 21 and tmin > 6 and dias_sin_lluvia_5 is true and estacion is not null
                            group by estacion 
                            order by estacion'''.format(estacion), conn)
    ventana_umbria = pd.read_sql('''
                            select estacion, count(*) as vu
                            from meteo_{}
                            where tmax < 21 and tmin > 6 and dias_sin_lluvia_8 is true and estacion is not null
                            group by estacion 
                            order by estacion'''.format(estacion), conn)

    meteo = pd.merge(dias_lluvia, dias_sin_lluvia_5, how='left')
    meteo = pd.merge(meteo, dias_sin_lluvia_8, how='left')
    meteo = pd.merge(meteo, nieve, how='left')
    meteo = pd.merge(meteo, heladas, how='left')
    meteo = pd.merge(meteo, ventana_solana, how='left')
    meteo = pd.merge(meteo, ventana_umbria, how='left')
    meteo = meteo.set_index('estacion')
    meteo = meteo.fillna(0)
    meteo = meteo.convert_dtypes()
    meteo.loc['media'] = meteo.mean(axis=0)
    meteo.index.name = 'Periodo'
    meteo.columns=['días lluvia', '5 días sin lluvia', '8 días sin lluvia', 'nieve', 'heladas', 'ventana solana', 'ventana umbría']
    datos_meteo.append({'estacion': estacion, 'meteo':meteo})
display(datos_meteo)

# CERRAR CONEXIÓN BASE DE DATOS

conn.close()

# EXPORTAR 

capitulos = ['introduccion', 'objetivos', 'ambito', 'requisitos', 'puestos', 'planificacion', 'elementos', 'juicio']

env = Environment(loader=FileSystemLoader('static/templates_vera'))
template =env.get_template('base.html')
output = template.render(
    zona = zona,
    severidad = severidad,
    condiciones_sinopticas = condiciones_sinopticas,
    tabla = tabla,
    catastro = catastro,
    forestales = forestales,
    capitulos = capitulos,
    rios = rios,
    datos_meteo = datos_meteo
)

with open('static/html/final.html', 'w') as f:
    f.write(output)

forestal   contacto
0  Arturo Diaz Herrero  620452991

[{'estacion': 'madrigal',
  'meteo':                    días lluvia 5 días sin lluvia 8 días sin lluvia nieve  \
  Periodo                                                                    
  invierno_2013_2014          55                30                15     1   
  invierno_2014_2015          39                53                30     3   
  invierno_2015_2016          41                41                22     0   
  invierno_2016_2017          40                47                27     1   
  invierno_2017_2018          37                54                33     0   
  invierno_2018_2019          46                37                19     1   
  invierno_2019_2020          52                34                17     1   
  media                  44.2857           42.2857           23.2857     1   
  
                     heladas ventana solana ventana umbría  
  Periodo                                                   
  invierno_2013_2014       3             11              6  